In [1]:
import requests
from selenium import webdriver as wbdv
import time
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# Data Extraction 

## Extracting all designers

In [2]:
all_designers_url="https://ap0cene.com/pages/ap0cene-designers-a-z"

In [3]:
all_designers_page=requests.get(all_designers_url)
soup=bs(all_designers_page.content, "html.parser")

In [4]:
all_designers_soup=soup.find_all(href=re.compile("https:\/\/ap0cene.com\/collections.+"))[2:]

In [5]:

df=pd.DataFrame()
designer_names=[]
designer_links=[]


for i,node in enumerate(all_designers_soup):
    designer_names.append(node.text)
    designer_links.append(node["href"])
    
df['designer_name']=designer_names
df['designer_page']=designer_links

#dropping invalide page
df.drop(21,inplace=True)

df.head(25)

,designer_name,designer_page
0,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...
1,Aephotika,https://ap0cene.com/collections/aephotika
2,Aiyana Monae,https://ap0cene.com/collections/aiyana-monae
3,Alexandra Zhukova,https://ap0cene.com/collections/alexandra-zhukova
4,Alisa Cayoo,https://ap0cene.com/collections/alisa-cayoo
5,Andagain,https://ap0cene.com/collections/andagain
6,Antispacetravel,https://ap0cene.com/collections/antispacetravel
7,Aporeei,https://ap0cene.com/collections/aporeei
8,Arezou,https://ap0cene.com/collections/arezou
9,Asterisk,https://ap0cene.com/collections/asterisk


In [6]:
#finding missing designer info

(df=='').sum()

designer_name    8
designer_page    0
dtype: int64

In [7]:
designers_with_name_missing=df[df['designer_name']=='']

In [8]:
designers_with_name_missing

,designer_name,designer_page
10,,https://ap0cene.com/collections/aporeei
26,,https://ap0cene.com/collections/dirt
37,,https://ap0cene.com/collections/ffluenzaa
38,,https://ap0cene.com/collections/freckle-farm
51,,https://ap0cene.com/collections/katy-brett-1
54,,https://ap0cene.com/collections/lapiin-lapiin
63,,https://ap0cene.com/collections/mikaela-marten...
110,,https://ap0cene.com/collections/xtinel


In [9]:
df.loc[0]["designer_name"]

'A Beautiful Orgasm'

In [10]:
#extract designer name from their personal page

def extract_designer_name(ind):
    url=df.loc[ind]["designer_page"]
    
    r=requests.get(url)
    soup=bs(r.content, "html.parser")
    designer_name=soup.find("h1", class_="font-heading text-lg").text
    
    return designer_name
    

In [11]:
#imputing missing names

for ind in designers_with_name_missing.index:
    try :
        df.loc[ind]["designer_name"]=extract_designer_name(ind)
    except:
        print(ind,"problem")

In [12]:
designers_with_name_missing.index

Int64Index([10, 26, 37, 38, 51, 54, 63, 110], dtype='int64')

In [13]:
#test again
(df=='').sum()

designer_name    0
designer_page    0
dtype: int64

In [14]:
#extracting designer description from their personal page
def extract_designer_description(url):
    r=requests.get(url)
    soup=bs(r.content, "html.parser")
    try :
        description=soup.find("div", class_="rte mt-4").text
        return description
    except:
        return "A designer."

In [15]:
df["designer_description"]=df["designer_page"].apply(extract_designer_description)

In [16]:
##ffs where are the probelms
##ok upon inspection the probelm was some of these designers have no description, 
# should i give them one? 
#absolutely not, because they can find more more info on their product description 
#thank you!


df[(df["designer_description"]=="A designer.")]

,designer_name,designer_page,designer_description
45,Jae Kim,https://ap0cene.com/collections/jae-kim,A designer.
52,Kerne.Milk,https://ap0cene.com/collections/vendors?q=Kern...,A designer.
58,Maran,https://ap0cene.com/collections/maran,A designer.
85,Shulian,https://ap0cene.com/collections/vendors?q=shulian,A designer.
92,Spirit Guide 9,https://ap0cene.com/collections/spirit-guide-9,A designer.
93,Stomachofyourdeadsoulmate,https://ap0cene.com/collections/stomachofyourd...,A designer.


In [18]:
df['designer_description']=df['designer_description'].apply(lambda x: x.replace('\n',''))
df_designer=df

In [19]:
df_designer.head(2)

,designer_name,designer_page,designer_description
0,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...
1,Aephotika,https://ap0cene.com/collections/aephotika,"As a designer, Katya views fashion as a form o..."


# Product Extraction

In [20]:
url='https://ap0cene.com/collections/yue-qing-wei-1'

In [21]:
df_designer.loc[0]['designer_page']

'https://ap0cene.com/collections/a-beautiful-orgasm'

In [22]:
#scrap the main page of ap0cene and return designer items

def designer_page_scrap(url):
    r=requests.get(url)
    soup=bs(r.content, "html.parser")
    names=[]
    items=[]
    descriptions=[]
    for i in soup.find_all("img", class_="responsive-image block absolute top-0 left-0 w-full h-full lazyload transition-opacity duration-200 ease-in-out w-full max-w-full h-auto" ):
        names.append(i['alt'])
        items.append('-'.join(i['alt'].lower().split()))
        
    return names,items,descriptions
        

In [23]:
scrapped=df_designer['designer_page'].apply(designer_page_scrap)

In [24]:
[scrapped.apply(lambda x:x[0]),scrapped.apply(lambda x:x[1])]

[0      [Infinity Pants, Spider Top, Cool Girl Denim, ...
 1      [Realm Beyond, Calcified Heart Necklace, Fresh...
 2      [Bucket Hat, Bonnet, Mosaic Crochet Halter, Be...
 3      [Machine Knitted Grunge Top, Hairy Pants, Wrap...
 4      [Transparent Piercing Necklace, Nice Girl Knit...
                              ...                        
 109    [Crater Dress, Yo+ Bag, Flood Top, Pollution C...
 110    [Artemis Wool Hat, Undine Jester Headpiece, Na...
 111    [Cut Out Lace Sleeve, Chestnut Cut Out Corset ...
 112    [Blob Chrome Nails, Organic Unknown, Hybrid, C...
 113    [Angelic Devil Jersey Top, Teddy Boots Tan, Fl...
 Name: designer_page, Length: 113, dtype: object,
 0      [infinity-pants, spider-top, cool-girl-denim, ...
 1      [realm-beyond, calcified-heart-necklace, fresh...
 2      [bucket-hat, bonnet, mosaic-crochet-halter, be...
 3      [machine-knitted-grunge-top, hairy-pants, wrap...
 4      [transparent-piercing-necklace, nice-girl-knit...
                      

In [62]:
df_product=pd.DataFrame()

In [63]:
df_product=pd.DataFrame(data={'product_name':scrapped.apply(lambda x:x[0])})

In [64]:
df_full=pd.concat([df_product,df_designer],axis=1)

In [65]:
df_full

,names,items,designer_name,designer_page,designer_description
0,"[Infinity Pants, Spider Top, Cool Girl Denim, ...","[infinity-pants, spider-top, cool-girl-denim, ...",A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...
1,"[Realm Beyond, Calcified Heart Necklace, Fresh...","[realm-beyond, calcified-heart-necklace, fresh...",Aephotika,https://ap0cene.com/collections/aephotika,"As a designer, Katya views fashion as a form o..."
2,"[Bucket Hat, Bonnet, Mosaic Crochet Halter, Be...","[bucket-hat, bonnet, mosaic-crochet-halter, be...",Aiyana Monae,https://ap0cene.com/collections/aiyana-monae,Aiyana Monae is a fiber exploration artist wor...
3,"[Machine Knitted Grunge Top, Hairy Pants, Wrap...","[machine-knitted-grunge-top, hairy-pants, wrap...",Alexandra Zhukova,https://ap0cene.com/collections/alexandra-zhukova,This collection is a study of materials and te...
4,"[Transparent Piercing Necklace, Nice Girl Knit...","[transparent-piercing-necklace, nice-girl-knit...",Alisa Cayoo,https://ap0cene.com/collections/alisa-cayoo,Alisa Cayoo is a multidisciplinary artist base...
...,...,...,...,...,...
109,"[Crater Dress, Yo+ Bag, Flood Top, Pollution C...","[crater-dress, yo+-bag, flood-top, pollution-c...",Yo+,https://ap0cene.com/collections/yo-1,Yo+ is a made-to-order knitwear and accessorie...
110,"[Artemis Wool Hat, Undine Jester Headpiece, Na...","[artemis-wool-hat, undine-jester-headpiece, na...",Xtinel,https://ap0cene.com/collections/xtinel,"Founded in 2021, Xtinel seeks to imbue a sense..."
111,"[Cut Out Lace Sleeve, Chestnut Cut Out Corset ...","[cut-out-lace-sleeve, chestnut-cut-out-corset-...",Yue Qing Wei,https://ap0cene.com/collections/yue-qing-wei-1,London-based designer Yue Qing Wei intimately ...
112,"[Blob Chrome Nails, Organic Unknown, Hybrid, C...","[blob-chrome-nails, organic-unknown, hybrid, c...",Yykcova,https://ap0cene.com/collections/yykcova,Started as an experiment to combat the art blo...


In [76]:
def get_product_page(url):
    response=requests.get(url)
    soup=bs(response.content,"html.parser")
    page_objects=soup.find_all("a",class_="increase-target")
    suffix=[]
    
    for ob in page_objects:
        suffix.append(ob["href"])
    
    return suffix

In [78]:
df_full["product_suffix"]=df_full["designer_page"].apply(get_product_page)

In [60]:
df_full.columns

Index(['names', 'items', 'designer_name', 'designer_page',
       'designer_description'],
      dtype='object')

In [79]:
df_full.head()

,names,items,designer_name,designer_page,designer_description,product_suffix
0,"[Infinity Pants, Spider Top, Cool Girl Denim, ...","[infinity-pants, spider-top, cool-girl-denim, ...",A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,[/collections/a-beautiful-orgasm/products/infi...
1,"[Realm Beyond, Calcified Heart Necklace, Fresh...","[realm-beyond, calcified-heart-necklace, fresh...",Aephotika,https://ap0cene.com/collections/aephotika,"As a designer, Katya views fashion as a form o...","[/collections/aephotika/products/realm-beyond,..."
2,"[Bucket Hat, Bonnet, Mosaic Crochet Halter, Be...","[bucket-hat, bonnet, mosaic-crochet-halter, be...",Aiyana Monae,https://ap0cene.com/collections/aiyana-monae,Aiyana Monae is a fiber exploration artist wor...,[/collections/aiyana-monae/products/bucket-hat...
3,"[Machine Knitted Grunge Top, Hairy Pants, Wrap...","[machine-knitted-grunge-top, hairy-pants, wrap...",Alexandra Zhukova,https://ap0cene.com/collections/alexandra-zhukova,This collection is a study of materials and te...,[/collections/alexandra-zhukova/products/machi...
4,"[Transparent Piercing Necklace, Nice Girl Knit...","[transparent-piercing-necklace, nice-girl-knit...",Alisa Cayoo,https://ap0cene.com/collections/alisa-cayoo,Alisa Cayoo is a multidisciplinary artist base...,[/collections/alisa-cayoo/products/transparent...


In [80]:
df_full=df_full.explode(['names', 'items','product_suffix']).reset_index(drop=True)
df_full

,names,items,designer_name,designer_page,designer_description,product_suffix
0,Infinity Pants,infinity-pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...
1,Spider Top,spider-top,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/spide...
2,Cool Girl Denim,cool-girl-denim,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/cool-...
3,Vermicelli Top,vermicelli-top,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/vermi...
4,Realm Beyond,realm-beyond,Aephotika,https://ap0cene.com/collections/aephotika,"As a designer, Katya views fashion as a form o...",/collections/aephotika/products/realm-beyond
...,...,...,...,...,...,...
858,Cargo Mushroom Skirt,cargo-mushroom-skirt,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/cargo-mushroom-...
859,Teddy Boots Brown,teddy-boots-brown,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/teddy-boots-brown
860,Peachy Gloss Boots,peachy-gloss-boots,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/peachy-gloss-bo...
861,Berry Gloss Boots,berry-gloss-boots,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/berry-gloss-fro...


In [81]:
df_full.loc[0]

names                                                      Infinity Pants
items                                                      infinity-pants
designer_name                                          A Beautiful Orgasm
designer_page           https://ap0cene.com/collections/a-beautiful-or...
designer_description    Lucky Truong is a self-taught designer based i...
product_suffix          /collections/a-beautiful-orgasm/products/infin...
Name: 0, dtype: object

In [82]:
#find the desingers with no products
df_missing=df_full[df_full.isnull().any(axis=1)]

missing_index=df_missing.index

df_missing

,names,items,designer_name,designer_page,designer_description,product_suffix
316,NaN,NaN,Freckle Farm,https://ap0cene.com/collections/freckle-farm,"""Freckle Farm is a reflection of my passion fo...",NaN
384,NaN,NaN,Julia Bartsch,https://ap0cene.com/collections/julia-bartsch,Julia Bartsch is a Parisian season-less jewelr...,NaN
385,NaN,NaN,Katy Brett,https://ap0cene.com/collections/katy-brett-1,Katy Brett is a sculptress who uses found mate...,NaN
448,NaN,NaN,Maran,https://ap0cene.com/collections/maran,A designer.,NaN
477,NaN,NaN,Mega Mikaela,https://ap0cene.com/collections/mikaela-marten...,MEGA MIKAELA is a knitwear label founded by Mi...,NaN
493,NaN,NaN,Mega Mikaela,https://ap0cene.com/collections/mikaela-marten...,MEGA MIKAELA is a knitwear label founded by Mi...,NaN
650,NaN,NaN,Shulian,https://ap0cene.com/collections/vendors?q=shulian,A designer.,NaN


In [83]:
df_full.loc[316]

names                                                                 NaN
items                                                                 NaN
designer_name                                                Freckle Farm
designer_page                https://ap0cene.com/collections/freckle-farm
designer_description    "Freckle Farm is a reflection of my passion fo...
product_suffix                                                        NaN
Name: 316, dtype: object

In [84]:
df_full=df_full.drop(missing_index).reset_index(drop=True)

In [101]:
df_full

,product_name,designer_name,designer_page,designer_description,product_suffix
0,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...
1,Spider Top,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/spide...
2,Cool Girl Denim,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/cool-...
3,Vermicelli Top,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/vermi...
4,Realm Beyond,Aephotika,https://ap0cene.com/collections/aephotika,"As a designer, Katya views fashion as a form o...",/collections/aephotika/products/realm-beyond
...,...,...,...,...,...
851,Cargo Mushroom Skirt,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/cargo-mushroom-...
852,Teddy Boots Brown,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/teddy-boots-brown
853,Peachy Gloss Boots,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/peachy-gloss-bo...
854,Berry Gloss Boots,Zemeta,https://ap0cene.com/collections/zemeta-1,Zemeta is a New York City based female designe...,/collections/zemeta-1/products/berry-gloss-fro...


In [105]:
#scrapping the website using selenium webdriver 

def product_url(product_suffix):
    url=f'https://ap0cene.com{product_suffix}'
    print(url)
    return url


In [106]:
df_full['product_page']=df_full.apply(lambda x: product_url(x['product_suffix']), axis=1)

https://ap0cene.com/collections/a-beautiful-orgasm/products/infinity-pants
https://ap0cene.com/collections/a-beautiful-orgasm/products/spider-top
https://ap0cene.com/collections/a-beautiful-orgasm/products/cool-girl-denim
https://ap0cene.com/collections/a-beautiful-orgasm/products/vermicelli-top-white
https://ap0cene.com/collections/aephotika/products/realm-beyond
https://ap0cene.com/collections/aephotika/products/calcified-heart-necklace
https://ap0cene.com/collections/aephotika/products/fresh-heart-belly-bar
https://ap0cene.com/collections/aephotika/products/lost-talisman
https://ap0cene.com/collections/aephotika/products/cephalopod-s-egg-necklace
https://ap0cene.com/collections/aephotika/products/warrior-s-heart-necklace
https://ap0cene.com/collections/aephotika/products/ocean-s-eye-pendant
https://ap0cene.com/collections/aephotika/products/celestial-guide-earrings
https://ap0cene.com/collections/aephotika/products/phosphorescent-slime-earrings
https://ap0cene.com/collections/aephot

In [107]:
#check if product page is valid

def url_response_check(product_page):
    status=(requests.get(product_page).ok)
    return status

In [108]:
df_full['product_page_response']=df_full['product_page'].apply(url_response_check)

In [110]:
df_full.to_csv("../data/df_full_response.csv")

In [114]:
#testing if there's any missing pages
df_missing=df_full[df_full['product_page_response']==False]

In [117]:
df_full.head()

,product_name,designer_name,designer_page,designer_description,product_suffix,product_page,product_page_response
0,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...,https://ap0cene.com/collections/a-beautiful-or...,True
1,Spider Top,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/spide...,https://ap0cene.com/collections/a-beautiful-or...,True
2,Cool Girl Denim,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/cool-...,https://ap0cene.com/collections/a-beautiful-or...,True
3,Vermicelli Top,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/vermi...,https://ap0cene.com/collections/a-beautiful-or...,True
4,Realm Beyond,Aephotika,https://ap0cene.com/collections/aephotika,"As a designer, Katya views fashion as a form o...",/collections/aephotika/products/realm-beyond,https://ap0cene.com/collections/aephotika/prod...,True


In [263]:
#scraping the image through finding their child 

def get_pic(url):
    response=requests.get(url)
    soup=bs(response.content,"html.parser")
    pic_objects=soup.find_all("div",class_="responsive-image-wrapper")
    pic_url=[]
    width=360
    for ob in pic_objects:
        pic_url.append(ob.find_all("img")[0]["data-src"].replace("{width}","360").replace("//"," http://")) 
    return pic_url





In [264]:
get_pic(df_full["product_page"][0])

[' http://cdn.shopify.com/s/files/1/0542/1803/1277/products/IMG_7317_360x.png?v=1654122108',
 ' http://cdn.shopify.com/s/files/1/0542/1803/1277/products/IMG_1579_360x.jpg?v=1654189133',
 ' http://cdn.shopify.com/s/files/1/0542/1803/1277/products/IMG_1577_360x.jpg?v=1654189133',
 ' http://cdn.shopify.com/s/files/1/0542/1803/1277/products/IMG_1578_360x.jpg?v=1654189133']

In [265]:
df_full["product_image_url"]=df_full["product_page"].apply(get_pic)

In [266]:
df_full=df_full.explode('product_image_url').reset_index(drop=True)

In [302]:
df_full["num"]=df_full.index

In [304]:
df_full.to_csv("../data/df_full_image_url.csv")

In [287]:
#save the images locally from the url

def save_images(index,url):
    r = requests.get(url, stream=True).content
    with open(f'../images/image_{index}.jpg', 'wb') as handler:
        handler.write(r)
    print('image saved locally')
    pass

/var/folders/7r/9jv1lnvj4b7b5xwwyx68yrwc0000gn/T/ipykernel_32148/2101194661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["num"]=df_test.index


,product_name,designer_name,designer_page,designer_description,product_suffix,product_page,product_page_response,product_image_url,num
0,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...,https://ap0cene.com/collections/a-beautiful-or...,True,http://cdn.shopify.com/s/files/1/0542/1803/12...,0
1,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...,https://ap0cene.com/collections/a-beautiful-or...,True,http://cdn.shopify.com/s/files/1/0542/1803/12...,1
2,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...,https://ap0cene.com/collections/a-beautiful-or...,True,http://cdn.shopify.com/s/files/1/0542/1803/12...,2
3,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...,https://ap0cene.com/collections/a-beautiful-or...,True,http://cdn.shopify.com/s/files/1/0542/1803/12...,3
4,Infinity Pants,A Beautiful Orgasm,https://ap0cene.com/collections/a-beautiful-or...,Lucky Truong is a self-taught designer based i...,/collections/a-beautiful-orgasm/products/infin...,https://ap0cene.com/collections/a-beautiful-or...,True,http://cdn.shopify.com/s/files/1/0542/1803/12...,4


In [289]:
save_images(1,df_test.loc[1]['product_image_url'])

image saved locally


In [301]:
df_test.apply(lambda x: save_images(df_full.index,x["product_image_url"]), axis=1)

image saved locally
image saved locally
image saved locally
image saved locally
image saved locally


0    None
1    None
2    None
3    None
4    None
dtype: object

In [16]:
for x,image in enumerate(images):
    r = requests.get(image, stream=True).content
    with open(f'images/image_{x}.jpg', 'wb') as handler:
            handler.write(r)
    
print('all images saved locally')

all images saved locally


In [17]:
images

['http://cdn.shopify.com/s/files/1/0542/1803/1277/products/kJ7OcwB19I_2048x2048.jpg?v=1648057231',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/XChyWB7wYF_2048x2048.jpg?v=1648057231',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/yiFg2oXMmk_2048x2048.jpg?v=1648057231',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/bEGnAsQmmA_2048x2048.jpg?v=1647045765',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/3NaMNkriya_2048x2048.jpg?v=1647045765',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/uFYtv1ZWmZ_2048x2048.jpg?v=1648065987',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/flVuTOBq2r_2048x2048.jpg?v=1648065969',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/mgKU3s3ok5_2048x2048.jpg?v=1648065969',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/e7Z83YVjZP_2048x2048.jpg?v=1648065969',
 'http://cdn.shopify.com/s/files/1/0542/1803/1277/products/xeinVZckST_2048x2048.jpg?v=1648065969',
 'http://c